# Data Preparation

In [42]:
import nltk 
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer
import warnings
warnings.filterwarnings("ignore")
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ASUS\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\ASUS\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

## Data Collection

In [80]:
metadata = pd.read_csv('electronics.csv')
metadata.head()

item_id  user_id  rating  timestamp model_attr                category  \
0        0        0       5  6/13/1999     Female  Portable Audio & Video   
1        0        1       5  6/14/1999     Female  Portable Audio & Video   
2        0        2       3  6/17/1999     Female  Portable Audio & Video   
3        0        3       1   7/1/1999     Female  Portable Audio & Video   
4        0        4       2   7/6/1999     Female  Portable Audio & Video   

  brand  year user_attr  split  
0   NaN  1999       NaN      0  
1   NaN  1999       NaN      0  
2   NaN  1999       NaN      0  
3   NaN  1999       NaN      0  
4   NaN  1999       NaN      0

In [81]:
#summarize of the data structure and content
metadata.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1048575 entries, 0 to 1048574
Data columns (total 10 columns):
 #   Column      Non-Null Count    Dtype 
---  ------      --------------    ----- 
 0   item_id     1048575 non-null  int64 
 1   user_id     1048575 non-null  int64 
 2   rating      1048575 non-null  int64 
 3   timestamp   1048575 non-null  object
 4   model_attr  1048575 non-null  object
 5   category    1048575 non-null  object
 6   brand       273310 non-null   object
 7   year        1048575 non-null  int64 
 8   user_attr   144408 non-null   object
 9   split       1048575 non-null  int64 
dtypes: int64(5), object(5)
memory usage: 80.0+ MB


In [82]:
#print dimensions of metadata, returns rows and columns
print('Rows x Columns : ', metadata.shape[0], 'x', metadata.shape[1])
#print all the columns name
print('\nFeatures : ', metadata.columns.tolist())
#print number of unique values
print('\nUnique values:')
print(metadata.nunique())

Rows x Columns :  1048575 x 10

Features :  ['item_id', 'user_id', 'rating', 'timestamp', 'model_attr', 'category', 'brand', 'year', 'user_attr', 'split']

Unique values:
item_id         9490
user_id       941861
rating             5
timestamp       5753
model_attr         3
category          10
brand             50
year              20
user_attr          2
split              3
dtype: int64


In [83]:
# Check for missing values in 'brand' and 'user_attr' columns
print("Missing values before cleaning:")
print(metadata[['brand', 'user_attr']].isnull().sum())

# Drop rows with missing values in 'brand' and 'user_attr' columns
metadata_cleaned = metadata.dropna(subset=['brand', 'user_attr'])

# Check for missing values in 'brand' and 'user_attr' columns after cleaning
print("Missing values after cleaning:")
print(metadata_cleaned[['brand', 'user_attr']].isnull().sum())

# Display the first few rows of the cleaned dataset
metadata_cleaned.head()

Missing values before cleaning:
brand        775265
user_attr    904167
dtype: int64
Missing values after cleaning:
brand        0
user_attr    0
dtype: int64


item_id  user_id  rating   timestamp   model_attr        category  \
131        7      131       4   6/15/2000       Female      Home Audio   
233       19      231       5  12/31/2000       Female  Camera & Photo   
235       14      233       5    1/2/2001  Female&Male  Camera & Photo   
259       14      257       5    2/4/2001  Female&Male  Camera & Photo   
271       14      269       5    3/2/2001  Female&Male  Camera & Photo   

       brand  year user_attr  split  
131  Philips  2001      Male      0  
233    Canon  2000      Male      0  
235    Kodak  2001    Female      0  
259    Kodak  2001    Female      0  
271    Kodak  2001      Male      0

In [73]:
#calculate mean of rating average column
#to show average rating on a scale of 5
calculate_rating = metadata['rating'].mean()
print(calculate_rating)

4.062871992942803


In [74]:
#calculated mean of rating average column after clean dataset
calculated_rating = metadata_cleaned['rating'].mean()
print(calculated_rating)

4.222041544271926


In [75]:
#print dimensions of metadata, returns rows and columns
print('Rows x Columns : ', metadata_cleaned.shape[0], 'x', metadata_cleaned.shape[1])

Rows x Columns :  38128 x 10


# Data Preprocessing

In [84]:
# Group the data by brand and calculate the mean rating for each group
mean_ratings = metadata_cleaned.groupby('brand')['rating'].mean()

# Sort the brands by mean rating in descending order
sorted_brands = mean_ratings.sort_values(ascending=False)

# Display the sorted brands and their mean ratings
print("Brands sorted by mean rating:")
print(sorted_brands)

Brands sorted by mean rating:
brand
Plemo           4.629252
Savage          4.612150
Tiamat          4.574468
ebasy           4.557823
Fintie          4.549693
Bose            4.526100
Apple           4.483531
Neewer          4.454849
Fujifilm        4.446688
Etre Jeune      4.442929
Garmin          4.356340
TaoTronics      4.346599
Canon           4.333766
Koolertron      4.326923
Gary Fong       4.311688
Olympus         4.307692
Panasonic       4.305816
Sony            4.302476
Toshiba         4.300395
EldHus          4.270492
Logitech        4.247078
Samsung         4.235455
Kensington      4.173469
Sennheiser      4.141159
Polaroid        4.140673
LSS             4.103933
XShields        4.069444
Nikon           4.063576
Skullcandy      4.043321
Cooper Cases    4.040000
HP              4.034043
Philips         4.033879
Mpow            4.030897
Uniden          4.027197
DURAGADGET      3.962963
Kodak           3.950370
ViewSonic       3.948310
Generic         3.915789
DBPOWER       

In [87]:
# Group the data by 'category' and 'user_attr' and count the number of occurrences in each group
grouped_data = metadata.groupby(['category', 'user_attr']).size()

# Sort the grouped data in descending order based on the count of occurrences
sorted_data = grouped_data.sort_values(ascending=False)

# Display the sorted data
print("Categories sorted by user_attr:")
print(sorted_data)

Categories sorted by user_attr:
category                 user_attr
Headphones               Male         20832
                         Female       20270
Computers & Accessories  Male         18715
                         Female       18691
Camera & Photo           Female       11148
                         Male         10897
Accessories & Supplies   Male          9333
                         Female        8117
Portable Audio & Video   Female        7368
                         Male          6801
Television & Video       Male          1902
Car Electronics & GPS    Male          1794
Television & Video       Female        1677
Home Audio               Male          1458
Car Electronics & GPS    Female        1192
Wearable Technology      Female        1122
Home Audio               Female        1017
Security & Surveillance  Male           724
                         Female         694
Wearable Technology      Male           656
dtype: int64
